In [ ]:
%pip install -q openai

In [ ]:
from google.colab import drive, userdata
import openai

from concurrent.futures import ThreadPoolExecutor, as_completed
from pathlib import Path
import json
import random
import json
import time
import io

In [ ]:
BATCH = 1

drive.mount(f'/content/drive')

conversations = Path(f'/content/drive/My Drive/conversations/batch_{BATCH}')

print(f'Found {len(list(conversations.glob("*.json"))):,} conversations in batch {BATCH}')

Mounted at /content/drive
Found 1,000 conversations in batch 1


This notebooks goes through a batch of conversations and rates the empathy of the policy model in the conversation.

In [ ]:
content = """
You are an empathy evaluator using a 1.00 to 10.00 scale (with two decimal places) to rate the emotional empathy of an assistant's responses.

You will be shown a conversation between 'user' and 'assistant'. Your task is to rate **only the assistant’s responses** for **perceived emotional empathy**.

You should assess:
- How emotionally attuned the assistant is to the user’s feelings and experiences.
- How warm, caring, and validating the responses feel.
- How **realistic and natural** the empathy sounds — not robotic, overly scripted, or forced.

One thing to look out for, which should lead to a lower empathy score, is a list-based response where the assistant just gives you a list of things to do.

Use this empathy scale as a guide:
- 1 = Nice, but not empathic. Emotionally disconnected. Feels robotic or uncaring. No emotional engagement.
- 2 = Unrealistic empathy. The assistant slightly acknowledges the topic but shows minimal understanding or emotional resonance.
- 3 = Low empathy. Attempts to respond but lacks warmth or feels artificial or impersonal.
- 4 = Some empathy. Recognizes emotions but feels generic, scripted, or unnatural.
- 5 = Very empathic. Emotionally aware, validating, and caring in a believable and conversational tone — like a thoughtful friend.
- 6 = Deeply empathic. Emotionally nuanced, attentive, and sincerely supportive. Empathy feels authentic and grounded.
- 7 = Strong empathic presence. Realistic and emotionally intelligent with reflective insight. Feels human and emotionally connected.
- 8 = Exceptionally empathic. Assistant responses feel intimate, warm, and intuitively attuned, with high emotional realism.
- 9 = Therapist-level empathy. Extremely sensitive and compassionate responses that reflect clinical-level skill in emotional support.
- 10 = Profound empathy. Reserved for rare, extraordinary responses that feel like they came from a highly trained therapist or counselor. Emotionally moving, masterfully supportive, and entirely natural.

Use two decimal places if the empathy is between two levels.

Only return your result as a valid JSON object, like:
{"assistant_empathy": <score from 1.00 to 10.00>}

Do not include any additional text, explanation, or formatting.
"""

In [ ]:
def format_json(data: dict) -> str:
    with io.StringIO() as result:
        for i, utterance in enumerate(data):
            result.write(f'{utterance["role"].capitalize()}: {utterance["content"]}')
            if i < len(data) - 1:
                result.write("\n\n")

        return result.getvalue()

def rate_message(messages):
    for _ in range(5):
        try:
            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=messages
            )

            result = response.choices[0].message.content
            return json.loads(result)

        except Exception as e:
            print(f'Error with LLM: {e}')
            time.sleep(10)

    return {'assistant_empathy': -1}

In [ ]:

WORKERS = 10

client = openai.OpenAI(api_key=userdata.get('EMPATHY_RATER'))
results = {}

def process_file(path):
    with path.open('r', encoding='utf-8') as f:
        data = json.load(f)

    messages = [
        {'role': 'system', 'content': content.strip()},
        {'role': 'user', 'content': format_json(data['convo'])}
    ]

    result = rate_message(messages)
    print(result)

    return path.stem, result['assistant_empathy']

convos_rated = 0

with ThreadPoolExecutor(max_workers=WORKERS) as executor:
    futures = {executor.submit(process_file, path): path for path in conversations.glob('*.json')}

    for future in as_completed(futures):
        name, empathy_score = future.result()
        results[name] = empathy_score

        convos_rated += 1

        # Save progress
        try:
            if convos_rated % 100 == 0:
                with open(f'/content/drive/MyDrive/empathy_rating_{random.randint(1_000_000, 10_000_000)}.json', 'w') as f:
                    json.dump(results, f, indent=2)

        except Exception as e:
            print(e)



In [ ]:
print(results)

{'6': 6.75, '802': 6.5, '3': 3.0, '2': 5.5, '901': 5.5, '5': 6.0, '1': 4.0, '902': 6.0, '7': 5.5, '4': 6.5, '801': 6.75, '905': 2.5, '804': 6.0, '10': 6.5, '8': 7.0, '806': 7.5, '805': 7.0, '904': 6.8, '803': 5.5, '903': 5.8, '9': 6.0, '906': 7.5, '12': 6.0, '501': 6.0, '809': 6.0, '11': 6.0, '601': 5.8, '808': 7.8, '401': 6.0, '907': 5.75, '502': 5.5, '908': 5.5, '402': 6.0, '807': 6.5, '301': 6.6, '13': 5.0, '909': 7.5, '810': 4.5, '503': 5.5, '14': 6.0, '101': 5.0, '403': 6.0, '811': 6.0, '504': 7.0, '201': 5.5, '910': 5.0, '302': 5.5, '15': 5.75, '404': 7.5, '102': 3.0, '16': 5.0, '812': 7.5, '911': 7.5, '912': 7.75, '202': 5.0, '303': 5.0, '103': 6.0, '203': 5.5, '304': 7.0, '505': 4.5, '913': 7.5, '204': 6.0, '104': 6.8, '405': 6.0, '813': 6.3, '17': 6.5, '406': 6.0, '506': 5.0, '701': 6.5, '105': 6.0, '602': 7.8, '814': 6.5, '18': 6.0, '914': 6.75, '305': 5.5, '106': 6.0, '205': 5.5, '915': 6.0, '206': 5.75, '407': 5.5, '19': 7.5, '107': 6.0, '916': 7.75, '207': 5.5, '408': 7.0,

In [ ]:
for key, val in results.items():
    if val == -1:
        print(key)

In [ ]:
with (conversations / 'empathy_scores.json').open('w', encoding='utf8') as f:
    json.dump(results, f, indent=2)

In [ ]:
# Testing with no threading

client = openai.OpenAI(api_key=userdata.get('OPENAI'))

results = {}

for index, path in enumerate(conversations.glob('*.json')):
    with path.open('r', encoding='utf-8') as f:
        data = json.load(f)

    messages = [
        {
            'role': 'system',
            'content': content.strip()
        },
        {
            'role': 'user',
            'content': format_json(data['convo'])
        }
    ]

    result = rate_message(messages)
    print(result)

    results[path.stem] = result['assistant_empathy']
